In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "order_id:{order_id}"),
    ]
)

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [3]:
classify_chain = order_change_cancel_prompt | model
classify_chain

In [8]:
store

{}

In [9]:
response = classify_chain_with_memory.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run ef48449c-509f-4df8-b6af-31695b79a1e7 not found for run c13b0f1c-4664-4c73-945e-669120249e2b. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 62, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-54fa0100-122a-49e1-8d8e-a4a9167451e1-0')

In [11]:
store["test_240516-1"].messages

[]

In [13]:
classify_chain_with_memory = RunnableWithMessageHistory(
    classify_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [17]:
response = classify_chain_with_memory.invoke(
    {"input": "주문 취송 원해",
     "order_id": None
     },
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run e622178c-abc3-4e06-9cf2-e3659b0bc959 not found for run 18247016-c184-4d60-bebf-f0f92a792bbb. Treating as a root run.


AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 62, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f54b5ef7-783e-4337-b609-65139baaeb06-0')

In [19]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취송 원해'),
 AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 62, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f54b5ef7-783e-4337-b609-65139baaeb06-0')]

# 사용자 동의 여부 생성 체인

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

classify_confirmation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in check_purpose.
            
            To make this determination, the following conditions must be met:
            1. There must be an AIMessage that asks for approval for the action specified in check_purpose, based on the specified order details.
            2. The user must have explicitly expressed consent in response to the AIMessage asking for approval of the action specified in check_purpose.
            
            Your response must be either 'yes' or 'no'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}\ncheck_purpose: {check_purpose}"),
    ]
)
classify_confirmation_prompt

ChatPromptTemplate(input_variables=['chat_history', 'check_purpose', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in check_purpose.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in check_purpose, based on the specified order details.\n            2. The 

In [31]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [32]:
classify_confirmation_prompt_chain = classify_confirmation_prompt | model

In [33]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_confirmation_prompt_chain_with_memory = RunnableWithMessageHistory(
    classify_confirmation_prompt_chain,
    get_session_history,
    input_messages_key="order_id",
    history_messages_key="chat_history",
)

In [34]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취송 원해'),
 AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 62, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f54b5ef7-783e-4337-b609-65139baaeb06-0')]

In [35]:
response = classify_confirmation_prompt_chain_with_memory.invoke(
    {"input": None,
     "order_id": 3,
     "check_purpose": "주문 취소"},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run adc63641-6ba4-4591-9f9e-e7b9cb27de96 not found for run f9b9022a-72d4-4567-b5a8-bce71bd9b5f8. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 182, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5f567cce-f4eb-46db-807e-bc2e05b9f762-0')

In [38]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취송 원해'),
 AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 62, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f54b5ef7-783e-4337-b609-65139baaeb06-0')]

In [39]:
response = classify_confirmation_prompt_chain_with_memory.invoke(
    {"input": "응",
     "order_id": 3,
     "check_purpose": "주문 취소"},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run dac11490-5b70-4bea-89ee-9c711be2ff3f not found for run 961f9105-6b0d-442b-a961-835847a0f11a. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='yes', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 183, 'total_tokens': 184}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4a8a894a-f349-4cc5-baf2-94a01e61f31c-0')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the check_purpose and queried_result.
            create a message to show the queried_result and ask for final confirmation considering check_purpose.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "check_purpose:{check_purpose}\nqueried_result:{queried_result}")
    ]
)

In [ ]:
generate_confirm_message_chain = generate_confirm_message_prompt | model

In [ ]:
def cancel_order(data: dict) -> dict:
    order_id = data["order_id"]
    
    return f"주문{order_id} 취소"

# 첫 번째 구성품

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "order_id:{order_id}"),
    ]
)

In [ ]:
classify_change_or_cancel_chain = order_change_cancel_prompt | model
classify_change_or_cancel_chain

In [ ]:
classify_change_or_cancel_chain_with_memory = RunnableWithMessageHistory(
    classify_change_or_cancel_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 세 번째 구성품

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the check_purpose and queried_result.
            create a message to show the queried_result and ask for final confirmation considering check_purpose.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "check_purpose:{check_purpose}\nqueried_result:{queried_result}")
    ]
)

In [ ]:
generate_confirm_message_chain = generate_confirm_message_prompt | model

In [ ]:
from langchain_core.runnables import RunnableLambda

def execution_confirmation_route(info):
    # from .chains import handle_inquiry_chain, handle_request_chain
    print("execution_confirmation_route 함수로 전달된 데이터 -> ", info)
    if "yes" in info["execution_confirmation"].content.lower():
        return RunnableLambda(cancel_order)
    else:
        return generate_confirm_message_chain